# 🔎 Análisis exploratorio del dataset Air Quality in Madrid (2001-2018)

Este proyecto utiliza el dataset “Air Quality in Madrid (2001-2018)” de Kaggle, que contiene registros diarios y horarios de diversos contaminantes recogidos por estaciones de monitoreo en Madrid. El objetivo principal es comparar la calidad del aire entre los años 2001 y 2018, analizando cómo han cambiado los niveles de contaminantes y las tendencias temporales a lo largo de la década.

El dataset ofrece suficiente cantidad de registros para realizar consultas significativas, incluyendo análisis por contaminante, estación y período temporal, sin requerir infraestructura de procesamiento especial. Esto permite explorar tendencias históricas, estacionales y anuales, facilitando una evaluación clara de los cambios en la calidad del aire durante una década clave en Madrid.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os

# Ruta de los archivos CSV por año
csvs_folder = r"data\csvs_per_year\csvs_per_year"
csv_files = sorted(glob.glob(os.path.join(csvs_folder, "madrid_*.csv")))

# Cargar todos los datasets en un diccionario por año
dfs = {}
for csv_path in csv_files:
    # Extraer el año del nombre del archivo
    year = int(os.path.basename(csv_path).split('_')[1].split('.')[0])
    df = pd.read_csv(csv_path)
    # Renombrar columna 'station' a 'id' si existe
    if 'station' in df.columns:
        df = df.rename(columns={'station': 'id'})
    # Renombrar 'NO' a 'NOx' solo si existe y es 2015 o posterior
    if 'NO' in df.columns and year >= 2015:
        df = df.rename(columns={'NO': 'NOx'})
    dfs[year] = df

# Cargar el archivo de estaciones
stations_csv = r"data\stations.csv"
stations = pd.read_csv(stations_csv)

# Unificar columnas comunes entre todos los años
common_cols = set(dfs[min(dfs.keys())].columns)
for df in dfs.values():
    common_cols = common_cols.intersection(set(df.columns))
common_cols = list(common_cols)

# Crear un DataFrame por año con columnas comunes y nombre de estación
dfs_common = {}
for year, df in dfs.items():
    df_common = df[common_cols].merge(stations[['id','name']], on='id', how='left')
    dfs_common[year] = df_common

# Mostrar ejemplo de todos los años
for year in sorted(dfs_common.keys()):
    print(f"=== Dataset {year} (columnas comunes con nombre de estación) ===")
    display(dfs_common[year].head())

=== Dataset 2001 (columnas comunes con nombre de estación) ===


,TCH,BEN,NMHC,PM10,O_3,EBE,SO_2,TOL,id,CO,NO_2,date,name
0,NaN,NaN,NaN,105.000000,34.529999,NaN,6.34,NaN,28079001,0.37,58.400002,2001-08-01 01:00:00,NaN
1,1.24,1.5,0.07,100.599998,42.160000,1.49,8.11,10.82,28079035,0.34,56.250000,2001-08-01 01:00:00,Pza. del Carmen
2,NaN,NaN,NaN,100.099998,46.310001,NaN,7.85,NaN,28079003,0.28,50.660000,2001-08-01 01:00:00,NaN
3,NaN,NaN,NaN,69.779999,40.650002,NaN,6.46,NaN,28079004,0.47,69.790001,2001-08-01 01:00:00,Pza. de España
4,NaN,NaN,NaN,75.180000,66.309998,NaN,8.80,NaN,28079039,0.39,22.830000,2001-08-01 01:00:00,Barrio del Pilar


=== Dataset 2018 (columnas comunes con nombre de estación) ===


,TCH,BEN,NMHC,PM10,O_3,EBE,SO_2,TOL,id,CO,NO_2,date,name
0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,28079004,0.3,29.0,2018-03-01 01:00:00,Pza. de España
1,1.41,0.5,0.02,5.0,52.0,0.2,3.0,0.8,28079008,0.3,40.0,2018-03-01 01:00:00,Escuelas Aguirre
2,NaN,0.4,NaN,NaN,NaN,0.2,NaN,1.1,28079011,NaN,41.0,2018-03-01 01:00:00,Avda. Ramón y Cajal
3,NaN,NaN,NaN,NaN,54.0,NaN,NaN,NaN,28079016,0.3,35.0,2018-03-01 01:00:00,Arturo Soria
4,NaN,NaN,NaN,NaN,49.0,NaN,3.0,NaN,28079017,NaN,27.0,2018-03-01 01:00:00,Villaverde


## 📊 Robert (análisis general y descriptivo)

- Promedio anual: ¿Cuál es la concentración media de cada contaminante? (tabla resumen).


- Mes con peor calidad del aire: ¿Qué mes tuvo la media más alta de NO₂? (gráfico de barras mensual).

- Distribución de O₃: ¿Cuál es el rango de concentraciones de O₃ más frecuente? (histograma).

- Picos diarios: ¿Qué día presentó la concentración más alta de PM10 en ese año?

## 🌍 David (comparaciones y correlaciones)

- Estacionalidad en un año: ¿Qué contaminante presenta mayores diferencias entre invierno y verano?

- Correlación: ¿Existe correlación entre los niveles diarios de NO₂ y CO en ese año?

- Laborables vs fines de semana: ¿Hay diferencias en los niveles de NO₂ promedio entre días laborables y fines de semana?

- Variación horaria: ¿A qué horas del día se concentran los picos de NO₂ en promedio durante el año? (curva horaria).

## 🌱 Alba (limpieza, estaciones y comparación)

- Valores nulos: ¿Qué porcentaje de valores faltan en cada contaminante y cómo tratarlos?

- Estaciones de medición: ¿Qué estación registró la mayor concentración media de NO₂? (gráfico de barras por estación).

- Comparación mensual: ¿Cómo evolucionó la concentración de PM10 a lo largo de los meses? (línea mensual).

- Contaminante predominante: ¿Cuál fue el contaminante con mayor concentración media?